In [1]:
# Setup scenario
from copy import deepcopy
from pathlib import Path

from constants import DATA_ENTITIES_DIR, DATA_HAZARDS_DIR, DATA_TEMP_DIR, REQUIREMENTS_DIR
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from climada.engine import CostBenefit, Impact, ImpactCalc
from climada.engine.cost_benefit import risk_aai_agg
from climada.entity import DiscRates, Entity, Exposures, LitPop
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet, ImpfTropCyclone
from climada.entity.impact_funcs.storm_europe import ImpfStormEurope
from climada.entity.impact_funcs.trop_cyclone import ImpfSetTropCyclone
from climada.entity.measures import Measure, MeasureSet
from climada.hazard import Hazard
from climada.util.api_client import Client

from costben.costben_handler import CostBenefitHandler
from entity.entity_handler import EntityHandler
from exposure.exposure_handler import ExposureHandler
from hazard.hazard_handler import HazardHandler
from impact.impact_handler import ImpactHandler

import handlers
from logger_config import LoggerConfig

logger = LoggerConfig(logger_types=["file"])

costben_handler = CostBenefitHandler()
entity_handler = EntityHandler()
exposure_handler = ExposureHandler()
hazard_handler = HazardHandler()
impact_handler = ImpactHandler()


# Available Exposure data types in CLIMADA API for Egypt/Thailand: ['litpop']
# Available Hazard data types in CLIMADA API for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', flood, 'tropical_cyclone']
# Available climate scenarios for hazard type river_flood/tropical_c in country Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
# Available time horizons for hazard type river_flood in country Egypt: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070']

country_name = "Egypt"
exposure_type = "litpop"  # Available exposure types for Egypt/Thailand: ['litpop']
hazard_type = "river_flood"  # Available hazard types for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', 'flood', 'tropical_cyclone']
scenario = "rcp26"  # Available scenarios for Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
time_horizon = "2030_2050"  # Available time horizons for Egypt/Thailand: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070']
annual_growth = 1.01

client = Client()

In [ ]:
import h5py
from scipy.io import whosmat, loadmat

# Keys
['centroid_ID', 'comment', 'date', 'datenum', 'dd', 'event_ID', 'event_count', 'filename', 'frequency', 
 'intensity', 'lat', 'lati', 'lon', 'long', 'matrix_density', 'mm', 'name', 'orig_event_count', 'orig_event_flag', 
 'orig_years', 'orig_yearset', 'peril_ID', 'reference_year', 'units', 'windfield_comment', 'yyyy']

def get_hazard_from_mat(file_path):
    with h5py.File(file_path, 'r') as file:
        hazard_data = file['hazard']
      hazard = Hazard()
        hazard.centroids = np.array(hazard_data['centroid_ID'])
        hazard.intensity = np.array(hazard_data['intensity']['data'])
        hazard.event_id = np.array(hazard_data['event_ID'])
        hazard.date = np.array(hazard_data['date'])
        hazard.frequency = np.array(hazard_data['frequency'])
        # match all the other attributes
        
        return hazard
    
def check_mat_version(file_path):
    try:
        # Attempt to load the .mat file metadata
        mat_info = whosmat(file_path)
        print(f".mat file version is 7.2 or earlier. Contents: {mat_info}")
    except NotImplementedError as e:
        print(".mat file is version 7.3 or later (HDF5 format).")

file_path = DATA_HAZARDS_DIR / 'hazard_Thailand_RCP85_gev_RP10.mat'
check_mat_version(file_path)

In [2]:
# Example
entity_present = entity_handler.get_entity_from_xlsx(
    DATA_ENTITIES_DIR / "entity_TODAY_THA_drought_tree_crops.xlsx"
)
entity_present.check()
entity_present.exposures.ref_year = 2024
entity_future = entity_handler.get_future_entity(entity_present, 2050, 1.02)
entity_future.check()

hazard_present = hazard_handler.get_hazard_from_mat(DATA_HAZARDS_DIR / "hazard_today.mat")
hazard_present.check()

hazard_future = hazard_handler.get_hazard_from_mat(DATA_HAZARDS_DIR / "hazardrcp45.mat")
hazard_future.check()

impfset = impact_handler.get_impact_function_set(201)
impfset.check()

impact_present = impact_handler.calculate_impact(entity_present.exposures, hazard_present, impfset)
impact_future = impact_handler.calculate_impact(entity_future.exposures, hazard_future, impfset)

costben = costben_handler.calculate_cost_benefit(
    hazard_present, entity_present, hazard_future, entity_future, 2050
)
costben_handler.plot_cost_benefit(costben)
costben_handler.plot_waterfall(
    costben, hazard_present, entity_present, hazard_future, entity_future
)

exposure_handler.generate_exposure_geojson(entity_future.exposures, "Thailand")
hazard_handler.generate_hazard_geojson(hazard_future, "Thailand")
impact_handler.generate_impact_geojson(impact_future, "Thailand")
# ax = costben_handler.plot_waterfall(costben, haz, ent, haz_future, ent_future)
# costben.plot_arrow_averted(ax)
# costben.plot_event_view((10, 15))
# costben.plot_waterfall_accumulated(haz, ent, ent_future)

2024-04-01 11:52:46,199 - climada.hazard.base - WARNING - Impact function id=201 has mdr(0) != 0.The mean damage ratio must thus be computed for all values ofhazard intensity including 0 which can be very time consuming.
2024-04-01 11:52:46,546 - climada.hazard.base - WARNING - Impact function id=201 has mdr(0) != 0.The mean damage ratio must thus be computed for all values ofhazard intensity including 0 which can be very time consuming.

Measure                  Cost (USD )    Benefit (USD )    Benefit/Cost
---------------------  -------------  ----------------  --------------
vegetation management     6.6887e+07                 0               0
risk transfer             2.1e+07                    0               0

--------------------  -  ------
Total climate risk:   0  (USD )
Average annual risk:  0  (USD )
Residual risk:        0  (USD )
--------------------  -  ------
Net Present Values


c:\Users\gkalomalos\Projects\unu\climada-unu\climada_env\lib\site-packages\climada\engine\cost_benefit.py:1087: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  axis.set_xlim(0, xy_lim[0])
c:\Users\gkalomalos\Projects\unu\climada-unu\climada_env\lib\site-packages\climada\engine\cost_benefit.py:463: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  axis.set_xlim(0, max(self.tot_climate_risk / norm_fact,


2024-04-01 11:52:57,885 - climada.hazard.base - WARNING - Return period 250.0 exceeds max. event return period.


In [2]:
# exposureEconomicDict: "tree_crops", "grass_crops", "wet_markets"
# exposureNonEconomicDict: "farmers", "buddhist_monks", "water_users", "roads"
# hazardDict: "flood", "drought", "heatwaves"

# 103	People (tree crops farmers)
# 104	People (grass crops farmers)
# 105	People (water)

haz = hazard_handler.get_hazard_from_mat(DATA_HAZARDS_DIR / "hazard_drought_THA_rcp45.mat")
ent = entity_handler.get_entity_from_xlsx(DATA_ENTITIES_DIR / "entity_TODAY_THA_drought_wet_markets.xlsx")
ent_future = entity_handler.get_future_entity(ent, 2040, 1.01)
# impfset = impact_handler.get_impact_function_set(201)
# imp = impact_handler.calculate_impact(exp, haz, impfset)

In [3]:
ent.exposures.gdf

,category_id,latitude,longitude,value,value_unit,deductible,cover,impf_
0,203,6.482921,101.133576,3.255056e+08,USD,0,3.255056e+08,203
1,203,6.741797,100.690933,4.222737e+07,USD,1,4.222737e+07,203
2,203,6.727371,100.063501,4.442609e+07,USD,2,4.442609e+07,203
3,203,7.029596,100.485500,1.826731e+08,USD,3,1.826731e+08,203
4,203,6.910759,100.740484,3.269429e+07,USD,4,3.269429e+07,203
...,...,...,...,...,...,...,...,...
361,203,19.909962,99.829930,7.995324e+06,USD,361,7.995324e+06,203
362,203,19.912674,99.213590,6.275333e+07,USD,362,6.275333e+07,203
363,203,19.964429,99.858936,1.127796e+07,USD,363,1.127796e+07,203
364,203,20.005470,100.041636,2.273613e+07,USD,364,2.273613e+07,203


In [4]:
ent_future.exposures.gdf

,category_id,latitude,longitude,value,value_unit,deductible,cover,impf_
0,203,6.482921,101.133576,4.092136e+08,USD,0,3.255056e+08,203
1,203,6.741797,100.690933,5.308669e+07,USD,1,4.222737e+07,203
2,203,6.727371,100.063501,5.585084e+07,USD,2,4.442609e+07,203
3,203,7.029596,100.485500,2.296499e+08,USD,3,1.826731e+08,203
4,203,6.910759,100.740484,4.110205e+07,USD,4,3.269429e+07,203
...,...,...,...,...,...,...,...,...
361,203,19.909962,99.829930,1.005143e+07,USD,361,7.995324e+06,203
362,203,19.912674,99.213590,7.889116e+07,USD,362,6.275333e+07,203
363,203,19.964429,99.858936,1.417823e+07,USD,363,1.127796e+07,203
364,203,20.005470,100.041636,2.858303e+07,USD,364,2.273613e+07,203


In [9]:
def get_future_entity(entity: Entity, future_year: int, aag: float) -> Entity:
    try:
        present_year = entity.exposures.ref_year
        entity_future = deepcopy(entity)
        entity_future.exposures.ref_year = future_year
        number_of_years = future_year - present_year + 1
        growth = aag**number_of_years
        entity_future.exposures.gdf["value"] = entity_future.exposures.gdf["value"] * growth
        return entity_future
    except Exception as e:
        logger.log("error", f"Failed to generate future entity: {e}")
        return None

ent_fut = get_future_entity(ent, 2040, 1.01)
ent_fut.exposures.gdf.value.sum()

17011234425.483553

In [6]:
print(f"Present value: {ent.exposures.gdf.value.sum()}")        #13531446739.366928
print(f"Future value:  {ent_future.exposures.gdf.value.sum()}") #17011234425.483553

# Electron
print(f"Electron value:{13666761206.760597}")


Present value: 13531446739.366928
Future value:  17011234425.483553
Electron value:13666761206.760597


In [ ]:
impact_func_101 = ImpactFunc(
    haz_type = "HW",
    id = 101,
    intensity= np.array([ 10,  15,  20,  25,  28,  31,  35,  40,  45,  48,  52,  56,  60, 64,  68,  71,  74,  78,  81,  85,  90,  95,  98, 101, 105, 110, 115, 120, 125, 130]),
    mdd= np.array([0.0212, 0.0429, 0.0651, 0.088, 0.1116, 0.1358, 0.1607, 0.1863, 0.2126, 0.2398, 0.2677, 0.2964, 0.326, 0.3564, 0.3877, 0.4199, 0.4531, 0.4872, 0.5223, 0.5584, 0.5955, 0.6337, 0.6729, 0.7133, 0.7548, 0.7975, 0.8413, 0.8863, 0.9326, 0.9801]),
    paa= np.ones(25),
    intensity_unit= "WSDI",
    name= "People (monks)"
)

impact_func_102 = ImpactFunc(
    haz_type= "HW",
    id= 102,
    intensity= np.array([10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130]),
    mdd= np.array([0.0117, 0.0237, 0.036, 0.0486, 0.0616, 0.075, 0.0887, 0.1029, 0.1174, 0.1324, 0.1478, 0.1637, 0.18, 0.1968, 0.2141, 0.2319, 0.2502, 0.269, 0.2884, 0.3083, 0.3288, 0.3499, 0.3716, 0.3939, 0.4168, 0.4404, 0.4646, 0.4894, 0.515, 0.5412]),
    paa= np.ones(25),
    intensity_unit= "WSDI",
    name= "People (students)"
)

impact_func_103 = ImpactFunc(
    haz_type= "D",
    id= 103,
    intensity= np.array([-3.5, -3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5]),
    mdd= np.array([0.6667, 0.6667, 0.3906, 0.2232, 0.1216, 0.06, 0.0227, 0, 0]),
    paa= np.ones(9),
    intensity_unit= "SPI",
    name= "People (tree crops farmers)"
)

impact_func_104 = ImpactFunc(
    haz_type= "D",
    id= 104,
    intensity= np.array([-3.5, -3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5]),
    mdd= np.array([1, 1, 1, 0.7365, 0.4013, 0.1981, 0.0748, 0, 0]),
    paa= np.ones(9),
    intensity_unit= "SPI",
    name= "People (grass crops farmers)"
)

impact_func_105 = ImpactFunc(
    haz_type= "D",
    id= 105,
    intensity= np.array([-3.5, -3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5]),
    mdd= np.array([1, 0.5871, 0.3362, 0.1925, 0.1102, 0.0631, 0.0361, 0.0207, 0.0119]),
    paa= np.ones(9),
    intensity_unit= "SPI",
    name= "People (unmet water demand)"
)

impact_func_201 = ImpactFunc(
    haz_type= "D",
    id= 201,
    intensity= np.array([-3.5, -3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5]),
    mdd= np.array([0.4667, 0.1867, 0.0706, 0.0332, 0.0216, 0.013, 0.0107, 0, 0]),
    paa= np.ones(9),
    intensity_unit= "SPI",
    name= "Tree crops"
)

impact_func_202 = ImpactFunc(
    haz_type= "D",
    id= 202,
    intensity= np.array([-3.5, -3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5]),
    mdd= np.array([0.6, 0.2, 0.15, 0.1, 0.0713, 0.0381, 0.0148, 0, 0]),
    paa= np.ones(9),
    intensity_unit= "SPI",
    name= "Grass crops"
)

impact_func_203 = ImpactFunc(
    haz_type= "D",
    id= 203,
    intensity= np.array([-3.5, -3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5]),
    mdd= np.array([0.7, 0.25, 0.18, 0.12, 0.0613, 0.0381, 0.0148, 0, 0]),
    paa= np.ones(9),
    intensity_unit= "SPI",
    name= "Markets"
)

impact_func_301 = ImpactFunc(
    haz_type= "FL",
    id= 301,
    intensity= np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]),
    mdd= np.array([0, 0, 0, 1, 1, 1, 1]),
    paa= np.ones(7),
    intensity_unit= "meter",
    name= "Mobility"
)

impact_func_401 = ImpactFunc(
    haz_type= "FL",
    id= 401,
    intensity= np.array([0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 3, 4, 5]),
    mdd= np.array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    paa= np.ones(12),
    intensity_unit= "meter",
    name= "Temple (people)"
)

impact_func_402 = ImpactFunc(
    haz_type= "FL",
    id= 402,
    intensity= np.array([0, 0.5, 1, 1.5, 2, 3, 4, 5, 6]),
    mdd= np.array([0, 0.3266, 0.4941, 0.6166, 0.7207, 0.8695, 0.9315, 0.9836, 1]),
    paa= np.ones(9),
    intensity_unit= "meter",
    name= "Houses (people)"
)

impact_fun_id	intensity	mdd	paa	mdr	peril_ID	Intensity_unit	name


101	10	2.12%	1	0.021174556	HW	WSDI	People (monks)
101	15	4.29%	1	0.04285507	HW	WSDI	People (monks)
101	20	6.51%	1	0.065130857	HW	WSDI	People (monks)
101	25	8.80%	1	0.088026137	HW	WSDI	People (monks)
101	28	11.16%	1	0.111565125	HW	WSDI	People (monks)
101	31	13.58%	1	0.135772039	HW	WSDI	People (monks)
101	35	16.07%	1	0.160671097	HW	WSDI	People (monks)
101	40	18.63%	1	0.186286514	HW	WSDI	People (monks)
101	45	21.26%	1	0.21264251	HW	WSDI	People (monks)
101	48	23.98%	1	0.2397633	HW	WSDI	People (monks)
101	52	26.77%	1	0.267673102	HW	WSDI	People (monks)
101	56	29.64%	1	0.296396134	HW	WSDI	People (monks)
101	60	32.60%	1	0.325956611	HW	WSDI	People (monks)
101	64	35.64%	1	0.356378753	HW	WSDI	People (monks)
101	68	38.77%	1	0.387686775	HW	WSDI	People (monks)
101	71	41.99%	1	0.419904895	HW	WSDI	People (monks)
101	74	45.31%	1	0.453057331	HW	WSDI	People (monks)
101	78	48.72%	1	0.487168298	HW	WSDI	People (monks)
101	81	52.23%	1	0.522262016	HW	WSDI	People (monks)
101	85	55.84%	1	0.5583627	HW	WSDI	People (monks)
101	90	59.55%	1	0.595494568	HW	WSDI	People (monks)
101	95	63.37%	1	0.633681838	HW	WSDI	People (monks)
101	98	67.29%	1	0.672948725	HW	WSDI	People (monks)
101	101	71.33%	1	0.713319449	HW	WSDI	People (monks)
101	105	75.48%	1	0.754818225	HW	WSDI	People (monks)
101	110	79.75%	1	0.797469271	HW	WSDI	People (monks)
101	115	84.13%	1	0.841296805	HW	WSDI	People (monks)
101	120	88.63%	1	0.886325042	HW	WSDI	People (monks)
101	125	93.26%	1	0.932578202	HW	WSDI	People (monks)
101	130	98.01%	1	0.9800805	HW	WSDI	People (monks)
102	10	1.17%	1	0.011692425	HW	WSDI	People (students)
102	15	2.37%	1	0.023664235	HW	WSDI	People (students)
102	20	3.60%	1	0.035964752	HW	WSDI	People (students)
102	25	4.86%	1	0.048607346	HW	WSDI	People (students)
102	28	6.16%	1	0.061605392	HW	WSDI	People (students)
102	31	7.50%	1	0.074972261	HW	WSDI	People (students)
102	35	8.87%	1	0.088721327	HW	WSDI	People (students)
102	40	10.29%	1	0.102865961	HW	WSDI	People (students)
102	45	11.74%	1	0.117419536	HW	WSDI	People (students)
102	48	13.24%	1	0.132395425	HW	WSDI	People (students)
102	52	14.78%	1	0.147807	HW	WSDI	People (students)
102	56	16.37%	1	0.163667634	HW	WSDI	People (students)
102	60	18.00%	1	0.179990699	HW	WSDI	People (students)
102	64	19.68%	1	0.196789569	HW	WSDI	People (students)
102	68	21.41%	1	0.214077614	HW	WSDI	People (students)
102	71	23.19%	1	0.231868209	HW	WSDI	People (students)
102	74	25.02%	1	0.250174725	HW	WSDI	People (students)
102	78	26.90%	1	0.269010536	HW	WSDI	People (students)
102	81	28.84%	1	0.288389013	HW	WSDI	People (students)
102	85	30.83%	1	0.308323529	HW	WSDI	People (students)
102	90	32.88%	1	0.328827458	HW	WSDI	People (students)
102	95	34.99%	1	0.34991417	HW	WSDI	People (students)
102	98	37.16%	1	0.371597039	HW	WSDI	People (students)
102	101	39.39%	1	0.393889438	HW	WSDI	People (students)
102	105	41.68%	1	0.416804739	HW	WSDI	People (students)
102	110	44.04%	1	0.440356314	HW	WSDI	People (students)
102	115	46.46%	1	0.464557536	HW	WSDI	People (students)
102	120	48.94%	1	0.489421778	HW	WSDI	People (students)
102	125	51.50%	1	0.514962412	HW	WSDI	People (students)
102	130	54.12%	1	0.54119281	HW	WSDI	People (students)
103	-3.5	66.67%	1	0.666666667	D	SPI	People (tree crops farmers)
103	-3	66.67%	1	0.666666667	D	SPI	People (tree crops farmers)
103	-2.5	39.06%	1	0.390609706	D	SPI	People (tree crops farmers)
103	-2	22.32%	1	0.223172696	D	SPI	People (tree crops farmers)
103	-1.5	12.16%	1	0.121617016	D	SPI	People (tree crops farmers)
103	-1	6.00%	1	0.060020382	D	SPI	People (tree crops farmers)
103	-0.5	2.27%	1	0.022660135	D	SPI	People (tree crops farmers)
103	0	0.00%	1	0	D	SPI	People (tree crops farmers)
103	0.5	0.00%	1	0	D	SPI	People (tree crops farmers)
104	-3.5	100.00%	1	1	D	SPI	People (grass crops farmers)
104	-3	100.00%	1	1	D	SPI	People (grass crops farmers)
104	-2.5	100.00%	1	1	D	SPI	People (grass crops farmers)
104	-2	73.65%	1	0.736469897	D	SPI	People (grass crops farmers)
104	-1.5	40.13%	1	0.401336152	D	SPI	People (grass crops farmers)
104	-1	19.81%	1	0.198067261	D	SPI	People (grass crops farmers)
104	-0.5	7.48%	1	0.074778446	D	SPI	People (grass crops farmers)
104	0	0.00%	1	0	D	SPI	People (grass crops farmers)
104	0.5	0.00%	1	0	D	SPI	People (grass crops farmers)
105	-3.5	100.00%	1	1	D	SPI	People (unmet water demand)
105	-3	58.71%	1	0.587063916	D	SPI	People (unmet water demand)
105	-2.5	33.62%	1	0.336175619	D	SPI	People (unmet water demand)
105	-2	19.25%	1	0.192507228	D	SPI	People (unmet water demand)
105	-1.5	11.02%	1	0.110237122	D	SPI	People (unmet water demand)
105	-1	6.31%	1	0.063126061	D	SPI	People (unmet water demand)
105	-0.5	3.61%	1	0.036148437	D	SPI	People (unmet water demand)
105	0	2.07%	1	0.0207	D	SPI	People (unmet water demand)
105	0.5	1.19%	1	0.011853625	D	SPI	People (unmet water demand)
201	-3.5	46.67%	1	0.466666667	D	SPI	Tree crops
201	-3	18.67%	1	0.186666667	D	SPI	Tree crops
201	-2.5	7.06%	1	0.070609706	D	SPI	Tree crops
201	-2	3.32%	1	0.033172696	D	SPI	Tree crops
201	-1.5	2.16%	1	0.021617016	D	SPI	Tree crops
201	-1	1.30%	1	0.013020382	D	SPI	Tree crops
201	-0.5	1.07%	1	0.010660135	D	SPI	Tree crops
201	0	0.00%	1	0	D	SPI	Tree crops
201	0.5	0.00%	1	0	D	SPI	Tree crops
202	-3.5	60.00%	1	0.6	D	SPI	Grass crops
202	-3	20.00%	1	0.2	D	SPI	Grass crops
202	-2.5	15.00%	1	0.15	D	SPI	Grass crops
202	-2	10.00%	1	0.1	D	SPI	Grass crops
202	-1.5	7.13%	1	0.071336152	D	SPI	Grass crops
202	-1	3.81%	1	0.038067261	D	SPI	Grass crops
202	-0.5	1.48%	1	0.014778446	D	SPI	Grass crops
202	0	0.00%	1	0	D	SPI	Grass crops
202	0.5	0.00%	1	0	D	SPI	Grass crops
203	-3.5	70.00%	1	0.7	D	SPI	Markets
203	-3	25.00%	1	0.25	D	SPI	Markets
203	-2.5	18.00%	1	0.18	D	SPI	Markets
203	-2	12.00%	1	0.12	D	SPI	Markets
203	-1.5	6.13%	1	0.061336152	D	SPI	Markets
203	-1	3.81%	1	0.038067261	D	SPI	Markets
203	-0.5	1.48%	1	0.014778446	D	SPI	Markets
203	0	0.00%	1	0	D	SPI	Markets
203	0.5	0.00%	1	0	D	SPI	Markets
301	0	0%	1	0	FL	meter	Mobility
301	0.1	0%	1	0	FL	meter	Mobility
301	0.2	0%	1	0	FL	meter	Mobility
301	0.3	100%	1	1	FL	meter	Mobility
301	0.4	100%	1	1	FL	meter	Mobility
301	0.5	100%	1	1	FL	meter	Mobility
301	0.6	100%	1	1	FL	meter	Mobility
401	0	0.00%	1	0.00%	FL	meter	Temple (people)
401	0.25	0.00%	1	0.00%	FL	meter	Temple (people)
401	0.5	100.00%	1	100.00%	FL	meter	Temple (people)
401	0.75	100.00%	1	100.00%	FL	meter	Temple (people)
401	1	100.00%	1	100.00%	FL	meter	Temple (people)
401	1.25	100.00%	1	100.00%	FL	meter	Temple (people)
401	1.5	100.00%	1	100.00%	FL	meter	Temple (people)
401	1.75	100.00%	1	100.00%	FL	meter	Temple (people)
401	2	100.00%	1	100.00%	FL	meter	Temple (people)
401	3	100.00%	1	100.00%	FL	meter	Temple (people)
401	4	100.00%	1	100.00%	FL	meter	Temple (people)
401	5	100.00%	1	100.00%	FL	meter	Temple (people)
402	0	0.00%	1	0.00%	FL	meter	Houses (people)
402	0.5	32.66%	1	32.66%	FL	meter	Houses (people)
402	1	49.41%	1	49.41%	FL	meter	Houses (people)
402	1.5	61.66%	1	61.66%	FL	meter	Houses (people)
402	2	72.07%	1	72.07%	FL	meter	Houses (people)
402	3	86.95%	1	86.95%	FL	meter	Houses (people)
402	4	93.15%	1	93.15%	FL	meter	Houses (people)
402	5	98.36%	1	98.36%	FL	meter	Houses (people)
402	6	100.00%	1	100.00%	FL	meter	Houses (people)


In [ ]:
# List DataTypeInfos
data_type_infos = client.list_data_type_infos()
exposure_data_types = [
    data_type_info.data_type
    for data_type_info in data_type_infos
    if data_type_info.data_type_group == "exposures"
]
hazard_data_types = [
    data_type_info.data_type
    for data_type_info in data_type_infos
    if data_type_info.data_type_group == "hazard"
]

print(
    f"Available Exposure data types in CLIMADA API for all countries:\n{exposure_data_types}")
print(
    f"Available Hazard data types in CLIMADA API for all countries:\n{hazard_data_types}")

In [ ]:
# Available country Exposures and Hazard data types
dataset_infos = client.list_dataset_infos(
    properties={
        "country_name": country_name,
    }
)

exposure_data_types = list(
    set(
        [
            dataset_info.data_type.data_type
            for dataset_info in dataset_infos
            if dataset_info.data_type.data_type_group == "exposures"
        ]
    )
)
hazard_data_types = list(
    set(
        [
            dataset_info.data_type.data_type
            for dataset_info in dataset_infos
            if dataset_info.data_type.data_type_group == "hazard"
        ]
    )
)

print(
    f"Available Exposure data types in CLIMADA API for {country_name}: {exposure_data_types}")
print(
    f"Available Hazard data types in CLIMADA API for {country_name}: {hazard_data_types}")

In [ ]:
# Available climate scenarios and time horizons for specific hazard type in countries Thailand and Egypt
if hazard_type == "river_flood" or "wildfire":
    available_scenarios = list(
        set(
            [
                dataset_info.properties["climate_scenario"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
    available_time_horizons = list(
        set(
            [
                dataset_info.properties["year_range"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
if hazard_type == "tropical_cyclone":
    available_scenarios = list(
        set(
            [
                dataset_info.properties["climate_scenario"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
    available_time_horizons = list(
        set(
            [
                dataset_info.properties.get("ref_year")
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
if hazard_type == "earthquake":
    available_scenarios = []
    available_time_horizons = []
if hazard_type == "flood":
    available_scenarios = []
    available_time_horizons = list(
        set(
            [
                dataset_info.properties["year_range"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )

print(
    f"Available climate scenarios for hazard type {hazard_type} in country {country_name}: {available_scenarios}"
)
print(
    f"Available time horizons for hazard type {hazard_type} in country {country_name}: {available_time_horizons}"
)

In [ ]:
# Available country Hazard DatasetInfos
hazard_dataset_infos = client.list_dataset_infos(
    properties={
        "data_type": "river_flood",
        "country_name": "Thailand",
        "climate_scenario": "rcp26",
        "year_range": "2030_2050",
    }
)
hazard_dataset_infos

In [ ]:
# Get Exposure
# Available Exposures: ['litpop']
exposure_present = exposure_handler.get_exposure(country_name)
if annual_growth > 1:
    exposure_future = exposure_handler.get_growth_exposure(
        exposure_present, annual_growth, 2040)

In [ ]:
# Get Hazard
# Available Hazards: ['river_flood', 'tropical_cyclone', 'wildfire', 'flood', 'earthquake']
hazard_present = hazard_handler.get_hazard(
    hazard_type,
    "historical",
    "1980_2000",
    country_name,
)

if scenario != "historical":
    hazard_future = hazard_handler.get_hazard(
        hazard_type,
        scenario,
        time_horizon,
        country_name,
    )

In [ ]:
# Get Impact
impact_function_set = impact_handler.calculate_impact_function_set(hazard_present)
impact_present = impact_handler.calculate_impact(exposure_present, hazard_present, impact_function_set)

if scenario != "historical":
    if annual_growth > 1:
        impact_future = impact_handler.calculate_impact(
            exposure_future, hazard_future, impact_function_set)
    impact_future = impact_handler.calculate_impact(
        exposure_present, hazard_future, impact_function_set)

In [ ]:
measure_set = costben_handler.get_measure_set_from_excel('RF')
discount_rates = costben_handler.get_discount_rates_from_excel()

In [ ]:
import pandas as pd
import sqlite3

class DataFrameSQLite:
    def __init__(self, db_path):
        self.db_path = db_path

    def save_dataframe(self, df, table_name):
        """
        Save a DataFrame to the SQLite database.
        
        :param df: DataFrame to save.
        :param table_name: Name of the table to save the DataFrame to.
        """
        with sqlite3.connect(self.db_path) as conn:
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"DataFrame saved to {table_name} table.")

    def read_dataframe(self, table_name):
        """
        Read a DataFrame from the SQLite database.
        
        :param table_name: Name of the table to read the DataFrame from.
        :return: DataFrame read from the database.
        """
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        return df

# Example usage
db_path = 'my_data.db'  # Path to your SQLite database file
df_sqlite = DataFrameSQLite(db_path)

# Assuming you have a DataFrame `df` to save
df_sqlite.save_dataframe(exp_gdf, 'exposure')

# To read the saved DataFrame from the database
df_read = df_sqlite.read_dataframe('exposure')
print(df_read)


In [ ]:
import pandas as pd
import sqlite3


class ExcelToSQLite:
    def __init__(self, db_path):
        self.db_path = db_path

    def read_excel_to_df(self, excel_path, sheet_name):
        """Reads a specified sheet from an Excel file into a DataFrame."""
        return pd.read_excel(excel_path, sheet_name=sheet_name)

    def adjust_df_columns(self, df, columns_to_keep=None):
        """Adjusts DataFrame columns based on the provided list. If None, keeps all columns."""
        if columns_to_keep is not None:
            df = df[columns_to_keep]
        return df

    def save_df_to_sqlite(self, df, table_name):
        """Saves a DataFrame to an SQLite table, appending data if the table already exists."""
        with sqlite3.connect(self.db_path) as conn:
            df.to_sql(table_name, conn, if_exists="append", index=False)

    def read_table(self, table_name):
        """Reads a table from SQLite database into a DataFrame."""
        with sqlite3.connect(self.db_path) as conn:
            return pd.read_sql(f"SELECT * FROM {table_name}", conn)


# Example usage
db_path = "climadera.db"  # Path to your SQLite database file
excel_path = DATA_ENTITIES_DIR / "3_entity_TODAY_THAI_D_USD_1_modified.xlsx"  # Path to your Excel file
excel_to_sqlite = ExcelToSQLite(db_path)

# Define the sheets and corresponding table names
sheets_tables = {
    "assets": "exposures",
    "impact_functions": "impact_functions",
    "measures": "measures",
    "discount": "discount_rates",
    "names": "names",
}

# Iterate over sheets and tables, read, adjust (if needed), and save to SQLite
for sheet, table in sheets_tables.items():
    df = excel_to_sqlite.read_excel_to_df(excel_path, sheet)

    # Here you can define which columns to keep for each table if needed, e.g.:
    # if table == 'exposures':
    #     columns_to_keep = ['Column1', 'Column2']
    #     df = excel_to_sqlite.adjust_df_columns(df, columns_to_keep)
    # else:
    #     df = excel_to_sqlite.adjust_df_columns(df)

    excel_to_sqlite.save_df_to_sqlite(df, table)

In [ ]:
import pandas as pd
import sqlite3


class ExcelToSQLite:
    def __init__(self, db_path):
        self.db_path = db_path

    def read_excel_and_save(self, excel_path, tabs_columns_mapping):
        """
        Read specified tabs from an Excel file and save them to SQLite database.

        :param excel_path: Path to the Excel file.
        :param tabs_columns_mapping: Dict mapping tab names to their columns adjustments.
        """
        with sqlite3.connect(self.db_path) as conn:
            for tab, columns in tabs_columns_mapping.items():
                df = pd.read_excel(excel_path, sheet_name=tab)

                # Adjust columns if specified
                if columns:
                    df = df[columns]

                df.to_sql(tab, conn, if_exists="append", index=False)
                print(f"Data from {tab} tab saved to {tab} table.")

    def read_table(self, table_name):
        """
        Read data from a specified table in the SQLite database.

        :param table_name: Name of the table to read data from.
        :return: DataFrame with the table data.
        """
        with sqlite3.connect(self.db_path) as conn:
            return pd.read_sql(f"SELECT * FROM {table_name}", conn)


# Example usage
db_path = "climadera.db"  # Path to your SQLite database file
excel_path = DATA_ENTITIES_DIR / "3_entity_TODAY_THAI_D_USD_1_modified.xlsx"  # Path to your Excel file
tabs_columns_mapping = {
    "assets": None,  # Specify columns as a list if you want to adjust them, or None to include all
    "impact_functions": None,
    "measures": None,
    "discount": None,
    "names": None,
}

excel_to_sqlite = ExcelToSQLite(db_path)
excel_to_sqlite.read_excel_and_save(excel_path, tabs_columns_mapping)

# Reading data from one of the tables
df_assets = excel_to_sqlite.read_table("assets")
print(df_assets)